# CWFS iterations vs Sonic Temperature StdDev

Craig Lage - 18-Sep-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
import pandas as pd
import os
os.environ["no_proxy"] += ",.consdb"
from lsst.summit.utils import ConsDbClient
from lsst_efd_client import EfdClient
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay


In [ ]:
client = ConsDbClient('http://consdb-pq.consdb:8080/consdb')
efd_client = EfdClient('usdf_efd')

In [ ]:
startDay = 20240625
endDay = 20240917
iterations = []
durations = []
sonic_temp_stds = []
dayObs = startDay
while dayObs <= endDay:
    try:
        data = client.query(f'SELECT img_type, obs_start from cdb_latiss.exposure where day_obs = {dayObs}')
    except:
        dayObs = calcNextDay(dayObs)
        continue
    if len(data) < 10:
        dayObs = calcNextDay(dayObs)
        continue
    cwfs = data[data['img_type'] == 'CWFS']    
    if len(cwfs) < 2:
        dayObs = calcNextDay(dayObs)
        continue
        
    n_cwfs = 0
    for i in range(len(data)):
        if data['img_type'].value[i] == 'CWFS':
            #print(data['img_type'].value[i], data['obs_start'].value[i])
            if n_cwfs == 0:
                start = Time(data['obs_start'].value[i], format='isot', scale='tai')
            n_cwfs += 1
            if data['img_type'].value[i+1] != 'CWFS':
                
                end = Time(data['obs_start'].value[i+1], format='isot', scale='tai')
                delta = end - start
                sonic = await efd_client.select_time_series('lsst.sal.ESS.airTurbulence', \
                                                        ['sonicTemperatureStdDev'],  start.utc, end.utc, index=205)
                if len(sonic) == 0:
                    continue
                iteration = int(n_cwfs / 2)
                if iteration < 2 or iteration > 5:
                    continue
                if delta.sec > 750:
                    continue
                iterations.append(iteration)
                durations.append(delta.sec)
                sonic_std = np.median(sonic['sonicTemperatureStdDev'].values)
                sonic_temp_stds.append(sonic_std)
                n_cwfs = 0
    dayObs = calcNextDay(dayObs)
print(f"{len(iterations)} data points total")

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,10))
plt.suptitle("Influence of dome seeing on number of CWFS iterations", fontsize = 18)
plt.subplot(2,2,1)
plt.scatter(iterations, sonic_temp_stds)
plt.ylabel("Gill anemometer sonic temperature StdDev")
plt.xlabel("Number of CWFS iterations")
plt.subplot(2,2,2)
mean_duration = np.sum(durations) / len(iterations)
plt.scatter(durations, sonic_temp_stds)
plt.text(220, 0.12, f"Mean duration:\n{mean_duration:.0f} seconds")
plt.ylabel("Gill anemometer sonic temperature StdDev")
plt.xlabel("CWFS duration (sec)")
iters = [2,3,4,5]
temp_stdDevs_mean = []
temp_stdDevs_std = []
for iter in iters:
    t_stds = []
    for i in range(len(iterations)):
        if iterations[i] == iter:
            t_stds.append(sonic_temp_stds[i])
    temp_stdDevs_mean.append(np.mean(t_stds))
    temp_stdDevs_std.append(np.std(t_stds))
plt.subplot(2,2,3)
plt.errorbar(iters, temp_stdDevs_mean,  temp_stdDevs_std, marker='x', capsize = 3)
plt.ylabel("Gill anemometer sonic temperature StdDev")
plt.xlabel("Number of CWFS iterations")
plt.text(2.2, 0.09, f"{len(iterations)} CWFS runs")    
plt.savefig("/home/c/cslage/u/AuxTel/cwfs/Iterations_vs_Dome_Seeing_18Sep24.png")